In [1]:
pip install pandas numpy requests yfinance ccxt ta tweepy google-trends-api

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 135.4 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b776d628993fa8c48993345dfde13988c413c18eead71e636cde59959d59edc0
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
  Created wheel for google-trends-api: filename=google_trends_api-2.2.6-py3-none-any.whl size=6144 sha256=9268c92d6d9fe620df3f80101672810f8ca7d86a6e7ff0cbe8df5c34016807d8
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/22/6e/6b/7c3a7b95d61ae0ccb3992af631d46d0120a768dd6a04414d51
  Created wheel for peewee: filename=peewee-3.17.9-cp311-cp311-linux

In [15]:
import yfinance as yf
import pandas as pd

# Definir criptos y rango de fechas
start_date = "2020-01-01"
end_date = "2025-02-28"

# Descargar datos de Yahoo Finance
df_btc = yf.download("BTC-USD", start=start_date, end=end_date, interval="1d")
df_eth = yf.download("ETH-USD", start=start_date, end=end_date, interval="1d")

# Agregar columna de fecha al inicio
df_btc.insert(0, "Date", df_btc.index)
df_eth.insert(0, "Date", df_eth.index)

# Seleccionar solo las columnas necesarias
df_btc = df_btc[["Date", "Open", "High", "Low", "Close", "Volume"]]
df_eth = df_eth[["Date", "Open", "High", "Low", "Close", "Volume"]]

# Resetear el índice para que "Date" sea una columna normal
df_btc = df_btc.reset_index(drop=True)
df_eth = df_eth.reset_index(drop=True)

print("Paso 1 completado: Datos de precios descargados correctamente con 'Date' al inicio.")
display(df_btc)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Paso 1 completado: Datos de precios descargados correctamente con 'Date' al inicio.


Price,Date,Open,High,Low,Close,Volume
Ticker,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095
...,...,...,...,...,...,...
1880,2025-02-23,96577.804688,96671.875000,95270.453125,96273.921875,16999478976
1881,2025-02-24,96277.960938,96503.453125,91371.742188,91418.171875,44046480529
1882,2025-02-25,91437.117188,92511.078125,86008.234375,88736.171875,92139104128


In [21]:
display(df_btc)

Price,Date,Open,High,Low,Close,Volume
Ticker,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095
...,...,...,...,...,...,...
1880,2025-02-23,96577.804688,96671.875000,95270.453125,96273.921875,16999478976
1881,2025-02-24,96277.960938,96503.453125,91371.742188,91418.171875,44046480529
1882,2025-02-25,91437.117188,92511.078125,86008.234375,88736.171875,92139104128


In [26]:
import requests
import pandas as pd

# 🔹 Función para obtener precios y volumen desde CryptoCompare
def get_crypto_data(symbol):
    url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym=USD&limit=2000"
    response = requests.get(url).json()

    if "Data" not in response or "Data" not in response["Data"]:
        print(f"Error: No hay datos para {symbol}")
        return None

    df = pd.DataFrame(response["Data"]["Data"])
    df["Date"] = pd.to_datetime(df["time"], unit="s")
    df = df[["Date", "close", "volumeto"]]
    df.columns = ["Date", f"{symbol}_Price_CryptoCompare", f"{symbol}_Volume_CryptoCompare"]

    return df

# 🔹 Obtener datos de BTC y ETH desde CryptoCompare
df_btc_crypto = get_crypto_data("BTC")
df_eth_crypto = get_crypto_data("ETH")

# 🔹 Asegurar que `df_btc` y `df_eth` no tienen MultiIndex
df_btc.columns = df_btc.columns.get_level_values(0)  # Quitar el MultiIndex
df_eth.columns = df_eth.columns.get_level_values(0)  # Quitar el MultiIndex

# 🔹 Resetear índice y asegurarse de que "Date" sea una columna normal
df_btc = df_btc.reset_index()
df_eth = df_eth.reset_index()

# 🔹 Convertir `Date` a `datetime64` en TODOS los DataFrames
df_btc["Date"] = pd.to_datetime(df_btc["Date"])
df_eth["Date"] = pd.to_datetime(df_eth["Date"])
df_btc_crypto["Date"] = pd.to_datetime(df_btc_crypto["Date"])
df_eth_crypto["Date"] = pd.to_datetime(df_eth_crypto["Date"])

# 🔹 Filtrar `df_btc_crypto` para que solo tenga fechas dentro del rango de `df_btc`
df_btc_crypto = df_btc_crypto[df_btc_crypto["Date"].between(df_btc["Date"].min(), df_btc["Date"].max())]
df_eth_crypto = df_eth_crypto[df_eth_crypto["Date"].between(df_eth["Date"].min(), df_eth["Date"].max())]

# 🔹 Hacer el merge correctamente (ahora sí, sin MultiIndex)
df_btc = df_btc.merge(df_btc_crypto, on="Date", how="left")
df_eth = df_eth.merge(df_eth_crypto, on="Date", how="left")

print("✅ Paso 2 completado: Datos de CryptoCompare agregados correctamente.")


✅ Paso 2 completado: Datos de CryptoCompare agregados correctamente.


In [28]:
# 🔹 Eliminar columnas innecesarias
df_btc.drop(columns=["level_0", "index"], inplace=True)
df_eth.drop(columns=["level_0", "index"], inplace=True)

print("✅ DataFrames limpios y listos para el paso 3.")

✅ DataFrames limpios y listos para el paso 3.


In [29]:
display(df_btc)

,Date,Open,High,Low,Close,Volume,BTC_Price_CryptoCompare,BTC_Volume_CryptoCompare
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,7189.94,8.598619e+07
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465,6965.72,1.873940e+08
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,7339.70,3.766904e+08
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275,7357.50,1.621069e+08
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095,7359.96,1.791477e+08
...,...,...,...,...,...,...,...,...
1880,2025-02-23,96577.804688,96671.875000,95270.453125,96273.921875,16999478976,96274.12,7.180437e+08
1881,2025-02-24,96277.960938,96503.453125,91371.742188,91418.171875,44046480529,91536.97,2.945423e+09
1882,2025-02-25,91437.117188,92511.078125,86008.234375,88736.171875,92139104128,88609.26,6.473415e+09
1883,2025-02-26,88638.890625,89286.250000,82131.898438,84347.023438,64597492134,84128.38,5.069738e+09


In [30]:
import ta

# 🔹 Función para agregar indicadores técnicos
def add_indicators(df, symbol):
    df[f"{symbol}_SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    df[f"{symbol}_EMA_20"] = ta.trend.ema_indicator(df["Close"], window=20)
    df[f"{symbol}_RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df[f"{symbol}_MACD"] = ta.trend.macd(df["Close"])

    return df

# 🔹 Aplicar indicadores a BTC y ETH
df_btc = add_indicators(df_btc, "BTC")
df_eth = add_indicators(df_eth, "ETH")

print("✅ Paso 3 completado: Indicadores técnicos agregados correctamente.")

✅ Paso 3 completado: Indicadores técnicos agregados correctamente.


In [32]:
display(df_btc)
print(df_btc.isna().sum())
print(df_eth.isna().sum())

,Date,Open,High,Low,Close,Volume,BTC_Price_CryptoCompare,BTC_Volume_CryptoCompare,BTC_SMA_50,BTC_EMA_20,BTC_RSI,BTC_MACD
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,7189.94,8.598619e+07,NaN,NaN,NaN,NaN
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465,6965.72,1.873940e+08,NaN,NaN,NaN,NaN
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,7339.70,3.766904e+08,NaN,NaN,NaN,NaN
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275,7357.50,1.621069e+08,NaN,NaN,NaN,NaN
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095,7359.96,1.791477e+08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1880,2025-02-23,96577.804688,96671.875000,95270.453125,96273.921875,16999478976,96274.12,7.180437e+08,98933.785156,97305.235503,45.487438,-803.149899
1881,2025-02-24,96277.960938,96503.453125,91371.742188,91418.171875,44046480529,91536.97,2.945423e+09,98795.849375,96744.562777,34.490698,-1173.411289
1882,2025-02-25,91437.117188,92511.078125,86008.234375,88736.171875,92139104128,88609.26,6.473415e+09,98529.011094,95981.858881,30.154477,-1664.078632
1883,2025-02-26,88638.890625,89286.250000,82131.898438,84347.023438,64597492134,84128.38,5.069738e+09,98277.497500,94873.779315,24.684970,-2379.672253


Date                         0
Open                         0
High                         0
Low                          0
Close                        0
Volume                       0
BTC_Price_CryptoCompare      0
BTC_Volume_CryptoCompare     0
BTC_SMA_50                  49
BTC_EMA_20                  19
BTC_RSI                     13
BTC_MACD                    25
dtype: int64
Date                         0
Open                         0
High                         0
Low                          0
Close                        0
Volume                       0
ETH_Price_CryptoCompare      0
ETH_Volume_CryptoCompare     0
ETH_SMA_50                  49
ETH_EMA_20                  19
ETH_RSI                     13
ETH_MACD                    25
dtype: int64


In [33]:
import requests

# 🔹 Función para obtener el Fear & Greed Index
def get_fear_greed():
    url = "https://api.alternative.me/fng/?limit=2000"
    response = requests.get(url).json()

    # Convertir en DataFrame
    data = response["data"]
    df_fng = pd.DataFrame(data)
    
    # Convertir fecha y ajustar formato
    df_fng["timestamp"] = pd.to_datetime(df_fng["timestamp"], unit="s")
    df_fng = df_fng.rename(columns={"timestamp": "Date", "value": "Fear_Greed_Index"})
    df_fng["Fear_Greed_Index"] = df_fng["Fear_Greed_Index"].astype(float)

    # Seleccionar columnas relevantes
    df_fng = df_fng[["Date", "Fear_Greed_Index"]]

    return df_fng

# 🔹 Obtener datos
df_fng = get_fear_greed()
display(df_fng)
print("✅ Fear & Greed Index descargado correctamente.")

/tmp/ipykernel_8342/3525605794.py:13: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_fng["timestamp"] = pd.to_datetime(df_fng["timestamp"], unit="s")


,Date,Fear_Greed_Index
0,2025-03-02,26.0
1,2025-03-01,20.0
2,2025-02-28,16.0
3,2025-02-27,10.0
4,2025-02-26,21.0
...,...,...
1995,2019-09-14,39.0
1996,2019-09-13,38.0
1997,2019-09-12,39.0
1998,2019-09-11,38.0


✅ Fear & Greed Index descargado correctamente.


In [34]:
# 🔹 Ajustar rango de fechas y granularidad
df_fng["Date"] = pd.to_datetime(df_fng["Date"])
df_fng = df_fng[df_fng["Date"].between("2020-01-01", "2025-02-28")]

# 🔹 Merge con nuestros DataFrames
df_btc = df_btc.merge(df_fng, on="Date", how="left")
df_eth = df_eth.merge(df_fng, on="Date", how="left")

print("✅ Fear & Greed Index agregado a df_btc y df_eth.")

✅ Fear & Greed Index agregado a df_btc y df_eth.


In [36]:
display(df_eth)

,Date,Open,High,Low,Close,Volume,ETH_Price_CryptoCompare,ETH_Volume_CryptoCompare,ETH_SMA_50,ETH_EMA_20,ETH_RSI,ETH_MACD,Fear_Greed_Index
0,2020-01-01,129.630661,132.835358,129.198288,130.802002,7935230330,130.57,2.043569e+08,NaN,NaN,NaN,NaN,37.0
1,2020-01-02,130.820038,130.820038,126.954910,127.410179,8032709256,127.05,2.876447e+08,NaN,NaN,NaN,NaN,39.0
2,2020-01-03,127.411263,134.554016,126.490021,134.171707,10476845358,134.24,4.515285e+08,NaN,NaN,NaN,NaN,38.0
3,2020-01-04,134.168518,136.052719,133.040558,135.069366,7430904515,134.20,2.537235e+08,NaN,NaN,NaN,NaN,38.0
4,2020-01-05,135.072098,139.410202,135.045624,136.276779,7526675353,135.50,3.022515e+08,NaN,NaN,NaN,NaN,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,2025-02-23,2764.031006,2850.610352,2746.773682,2821.310303,20493767269,2819.92,1.058360e+09,3039.410557,2776.931975,49.703881,-81.928377,50.0
1881,2025-02-24,2820.456299,2839.010010,2478.899414,2513.821289,29127380459,2513.65,2.290917e+09,3017.004907,2751.873814,37.079125,-93.698728,49.0
1882,2025-02-25,2514.210938,2529.669189,2326.632080,2493.592529,40044790246,2492.48,3.220072e+09,2993.104531,2727.275597,36.423673,-103.466411,25.0
1883,2025-02-26,2493.381592,2503.422119,2255.054932,2331.452637,28815111323,2333.12,2.258691e+09,2972.102036,2689.578172,31.601678,-122.874286,21.0


In [41]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


In [46]:
from pytrends.request import TrendReq
import pandas as pd
import time

# Función para descargar datos de Google Trends con granularidad diaria
def get_google_trends_daily(keywords, start_year=2020, end_year=2025):
    pytrends = TrendReq(hl='es', tz=0)
    df_list = []

    for year in range(start_year, end_year):
        for month in [1, 7]:  # Partimos en intervalos de 6 meses
            start_date = f"{year}-{month:02d}-01"
            if month == 1:
                end_date = f"{year}-06-30"
            else:
                end_date = f"{year}-12-31"
            
            timeframe = f"{start_date} {end_date}"
            print(f"📥 Descargando Google Trends de {timeframe}...")

            pytrends.build_payload(keywords, timeframe=timeframe, geo="")
            df_temp = pytrends.interest_over_time()

            if not df_temp.empty:
                df_temp.drop(columns=['isPartial'], inplace=True, errors='ignore')
                df_list.append(df_temp)

            # Evitar bloqueo por Google
            time.sleep(5)

    if df_list:
        df_trends = pd.concat(df_list)
        df_trends.reset_index(inplace=True)
        return df_trends
    else:
        return None

# 🔹 Descargar los datos con granularidad diaria
df_trends_daily = get_google_trends_daily(["Bitcoin", "Ethereum"], 2020, 2025)

# 🔹 Guardar en CSV si se logró obtener
if df_trends_daily is not None:
    df_trends_daily.to_csv("google_trends_daily.csv", index=False)
    print("✅ Datos de Google Trends guardados en 'google_trends_daily.csv'")
else:
    print("❌ No se pudieron obtener datos diarios desde Google Trends.")

❌ No se pudieron obtener datos diarios desde Google Trends.


In [52]:
from pytrends.request import TrendReq
import pandas as pd
import time

def get_google_trends_january_2025(keywords):
    pytrends = TrendReq(hl='es', tz=0)
    
    start_date = "2025-01-01"
    end_date = "2025-01-31"
    timeframe = f"{start_date} {end_date}"
    
    print(f"📥 Descargando Google Trends de {timeframe}...")

    try:
        pytrends.build_payload(keywords, timeframe=timeframe, geo="")
        df_temp = pytrends.interest_over_time()

        if not df_temp.empty:
            df_temp.drop(columns=['isPartial'], inplace=True, errors='ignore')
            df_temp.reset_index(inplace=True)
            return df_temp
        
    except Exception as e:
        print(f"⚠️ Error en {timeframe}: {e}")
    
    return None

# 🔹 Intentar descargar enero 2025
df_trends_jan_2025 = get_google_trends_january_2025(["Bitcoin", "Ethereum"])

# 🔹 Si hay datos, guardarlos y agregarlos al dataset anterior
if df_trends_jan_2025 is not None:
    df_trends_jan_2025.to_csv("google_trends_jan_2025.csv", index=False)
    print("✅ Datos de Google Trends enero 2025 guardados en 'google_trends_jan_2025.csv'")
else:
    print("❌ No se obtuvieron datos para enero de 2025.")


📥 Descargando Google Trends de 2025-01-01 2025-01-31...
✅ Datos de Google Trends enero 2025 guardados en 'google_trends_jan_2025.csv'


In [54]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [57]:
import requests
import pandas as pd
import time

# 🔹 Configuración de API y filtros
API_KEY = "22d881bb7ed9132287a966b9ce2e87924842f77b"
BASE_URL = f"https://cryptopanic.com/api/v1/posts/?auth_token={API_KEY}"
LIMIT_PAGES = 10  # Solo 10 páginas para analizar estructura

def get_news(coin, pages=LIMIT_PAGES):
    all_news = []
    
    for page in range(1, pages + 1):
        print(f"📥 Obteniendo noticias de {coin}, página {page}")

        url = f"{BASE_URL}&currencies={coin}&page={page}"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"⚠️ Error en la página {page}: {response.status_code}")
            break
        
        data = response.json()
        
        if "results" not in data or not data["results"]:
            print(f"⚠️ No hay más noticias en la página {page}. Deteniendo...")
            break
        
        for item in data["results"]:
            all_news.append({
                "date": item["created_at"],
                "title": item["title"],
                "source": item["source"]["title"] if "source" in item else None,
                "url": item["url"],
                "currencies": ", ".join([c["code"] for c in item.get("currencies", [])])
            })
        
        # 🔹 Guardamos cada 5 páginas por si se necesita pausar
        if page % 5 == 0:
            df_temp = pd.DataFrame(all_news)
            df_temp.to_csv(f"crypto_news_{coin}.csv", index=False)
            print(f"✅ Noticias hasta la página {page} guardadas en 'crypto_news_{coin}.csv'")
        
        time.sleep(2)  # Evitamos saturar la API
    
    return pd.DataFrame(all_news)

# 🔹 Obtener noticias de BTC y ETH
df_btc_news = get_news("BTC")
df_eth_news = get_news("ETH")

# 🔹 Guardamos todo al final
df_btc_news.to_csv("crypto_news_BTC.csv", index=False)
df_eth_news.to_csv("crypto_news_ETH.csv", index=False)
print("✅ Noticias guardadas correctamente en CSV.")

📥 Obteniendo noticias de BTC, página 1
📥 Obteniendo noticias de BTC, página 2
📥 Obteniendo noticias de BTC, página 3
📥 Obteniendo noticias de BTC, página 4
📥 Obteniendo noticias de BTC, página 5
✅ Noticias hasta la página 5 guardadas en 'crypto_news_BTC.csv'
📥 Obteniendo noticias de BTC, página 6
📥 Obteniendo noticias de BTC, página 7
📥 Obteniendo noticias de BTC, página 8
📥 Obteniendo noticias de BTC, página 9
📥 Obteniendo noticias de BTC, página 10
✅ Noticias hasta la página 10 guardadas en 'crypto_news_BTC.csv'
📥 Obteniendo noticias de ETH, página 1
📥 Obteniendo noticias de ETH, página 2
📥 Obteniendo noticias de ETH, página 3
📥 Obteniendo noticias de ETH, página 4
📥 Obteniendo noticias de ETH, página 5
✅ Noticias hasta la página 5 guardadas en 'crypto_news_ETH.csv'
📥 Obteniendo noticias de ETH, página 6
📥 Obteniendo noticias de ETH, página 7
📥 Obteniendo noticias de ETH, página 8
📥 Obteniendo noticias de ETH, página 9
📥 Obteniendo noticias de ETH, página 10
✅ Noticias hasta la págin

In [61]:
import requests
import pandas as pd

# URL de la API de SentiCrypt
url = "https://api.senticrypt.com/v2/all.json"

# 🔹 Obtener los datos de la API
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    
    # 🔹 Convertir en DataFrame
    df_sentiment = pd.DataFrame(data)

    # 🔹 Filtrar fechas (ajustar según las fechas del dataset de precios)
    df_sentiment['date'] = pd.to_datetime(df_sentiment['date'])
    start_date = "2020-01-01"
    end_date = "2025-03-02"  # Última fecha que tenemos en otros datos

    df_sentiment = df_sentiment[(df_sentiment['date'] >= start_date) & (df_sentiment['date'] <= end_date)]

    # 🔹 Seleccionar solo las columnas útiles
    df_sentiment = df_sentiment[['date', 'mean']]
    df_sentiment.rename(columns={'mean': 'BTC_Sentiment'}, inplace=True)

    # 🔹 Unir con precios
    df_btc_sentiment = df_btc.merge(df_sentiment, left_on="Date", right_on="date", how="left")
    df_btc_sentiment.drop(columns=['date'], inplace=True)  # Ya tenemos "Date", eliminamos la duplicada

    # 🔹 Guardar dataset final
    df_btc_sentiment.to_csv("df_btc_sentiment.csv", index=False)
    print("✅ Sentimiento agregado y datos guardados en 'df_btc_sentiment.csv'")
    
else:
    print(f"⚠️ Error al obtener los datos: {response.status_code}")


✅ Sentimiento agregado y datos guardados en 'final_dataset.csv'


In [62]:
import pandas as pd

# 🔹 Cargar los datasets
df_btc = pd.read_csv("df_btc_sentiment.csv")  # BTC ya tiene sentimiento
df_eth = pd.read_csv("df_eth.csv")  # ETH sin sentimiento
df_trends = pd.read_csv("google_trends_daily.csv")  # Google Trends para ambos

# 🔹 Convertir fechas a datetime para evitar inconsistencias
df_btc['Date'] = pd.to_datetime(df_btc['Date'])
df_eth['Date'] = pd.to_datetime(df_eth['Date'])
df_trends['date'] = pd.to_datetime(df_trends['date'])

# 🔹 Unir BTC con Trends
df_btc = df_btc.merge(df_trends[['date', 'Bitcoin']], left_on="Date", right_on="date", how="left")
df_btc.rename(columns={"Bitcoin": "BTC_Trends"}, inplace=True)
df_btc.drop(columns=["date"], inplace=True)

# 🔹 Unir ETH con Trends
df_eth = df_eth.merge(df_trends[['date', 'Ethereum']], left_on="Date", right_on="date", how="left")
df_eth.rename(columns={"Ethereum": "ETH_Trends"}, inplace=True)
df_eth.drop(columns=["date"], inplace=True)

# 🔹 Guardar los datasets finales
df_btc.to_csv("btc_final.csv", index=False)
df_eth.to_csv("eth_final.csv", index=False)

print("✅ BTC y ETH con Google Trends guardados en 'btc_final.csv' y 'eth_final.csv'")

✅ BTC y ETH con Google Trends guardados en 'btc_final.csv' y 'eth_final.csv'
